In [ ]:
import numpy as np
from keras.applications.vgg16 import VGG16
import keras
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.utils import to_categorical
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data = np.load("../Hackdata/data.npy")
labels = np.load("../Hackdata/labels.npy")

print data.shape
print labels.shape

In [ ]:
labels = to_categorical(labels)
print labels.shape

In [ ]:
def preprocess_input(image):
    image = image[:, :, :, ::-1] #RGB to BGR
    image[:, :, :, 0] -= 103.939 
    image[:, :, :, 1] -= 116.779
    image[:, :, :, 2] -= 123.68
    return image

data = preprocess_input(data)

In [ ]:
split = int(0.80 * data.shape[0])
x_train = data[:split, :, :, :]
x_test = data[split:, :, :, :]

y_train = labels[:split]
y_test = labels[split:]

print x_train.shape, y_train.shape
print x_test.shape, y_test.shape

In [ ]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))

In [ ]:
x = Flatten()(base_model.output)
x = Dense(512)(x)
x = Dropout(0.5)(x)
x = Activation('relu')(x)
x = Dense(6)(x)
x = Activation('softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

adam = Adam(lr=0.00003)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

In [ ]:
for ix in range(19):
    model.layers[ix].trainable = False

model.summary()

In [ ]:
hist = model.fit(x_train, y_train, shuffle=True, batch_size=16, epochs=2, validation_data=(x_test, y_test))

In [ ]:
model.save_weights("../Hackdata/bottleneck.h5")

In [ ]:
model.layers

In [ ]:
plt.figure(0)
plt.imshow(x_train[500])
plt.show()

In [ ]:
first_block = Model(inputs=model.input, outputs=model.layers[2].output)
first_block.summary()

In [ ]:
out = first_block.predict(x_train[500, :, :, :].reshape((-1, 224, 224, 3)))
print out.shape

plt.figure(figsize=(224,224))
for ix in range(out.shape[3]):
    plt.subplot(8, 8, ix+1)
    plt.axis('off')
    plt.imshow(out[:, :, :, ix].reshape((224, 224)))
plt.show()

In [ ]:
second_last_block = Model(inputs=model.input, outputs=model.layers[13].output)
second_last_block.summary()

In [ ]:
out = second_last_block.predict(x_train[500, :, :, :].reshape((-1, 224, 224, 3)))
print out.shape

plt.figure(figsize=(28,28))
for ix in range(out.shape[3]/8):
    plt.subplot(8, 8, ix+1)
    plt.axis('off')
    plt.imshow(out[:, :, :, ix].reshape((28, 28)))
plt.show()

In [ ]:
last_block = Model(inputs=model.input, outputs=model.layers[17].output)
last_block.summary()

In [ ]:
out = last_block.predict(x_train[500, :, :, :].reshape((-1, 224, 224, 3)))
print out.shape

plt.figure(figsize=(14,14))
for ix in range(out.shape[3]/8):
    plt.subplot(8, 8, ix+1)
    plt.axis('off')
    plt.imshow(out[:, :, :, ix].reshape((14, 14)))
plt.show()